In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
def generate_dummy_images(num_samples=1000, image_size=(64, 64)):
    X = []
    y = []
    for _ in range(num_samples):
        # Malware: noisy/random pixel data
        img = np.random.randint(0, 256, size=image_size).astype('uint8')
        X.append(img)
        y.append(1)

        # Benign: smoother pixel data
        benign_img = np.random.normal(loc=128, scale=30, size=image_size).clip(0, 255).astype('uint8')
        X.append(benign_img)
        y.append(0)
    X = np.array(X).reshape(-1, image_size[0], image_size[1], 1) / 255.0
    y = np.array(y)
    return X, y

In [ ]:
def build_cnn(input_shape):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_cnn(X.shape[1:])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)
    preds = (model.predict(X_test) > 0.5).astype(int)
    print("\nClassification Report:\n", classification_report(y_test, preds))
    model.save("models/malware_cnn_model.h5")
    print("Model saved.")
   
  

In [ ]:
if __name__ == "__main__":
    X, y = generate_dummy_images()
    train_model(X, y)